In [ ]:
#S-Box Layer
s_box=[0xC,0x5,0x6,0xB,0x9,0x0,0xA,0xD,0x3,0xE,0xF,0x8,0x4,0x7,0x1,0x2]

# Rotate left: 0b1001 --> 0b0011
rol = lambda val, r_bits, max_bits: \
    (val << r_bits%max_bits) & (2**max_bits-1) | \
    ((val & (2**max_bits-1)) >> (max_bits-(r_bits%max_bits)))

# Rotate right: 0b1001 --> 0b1100
ror = lambda val, r_bits, max_bits: \
    ((val & (2**max_bits-1)) >> r_bits%max_bits) | \
    (val << (max_bits-(r_bits%max_bits)) & (2**max_bits-1))

def genRoundKeys(key, rounds=32):
    roundkeys = []
    for i in range(1, rounds+1):
        '''
        At round i the 64-bit round key Ki = k63 k62 . . . k0 consists 
        of the 64 leftmost bits of the current contents of register K:
        K_i = k63 k62 ...k0 = k79 k78 ...k16      
        '''
        roundkeys.append(key >> 16)

        # After extracting Ki, the key register K = k79k78 . . . k0 is updated:

        # the key register is rotated by 61 bit positions to the left
        key = rol(key, 61, 80)

        # the left-most four bits are passed through the present S-box
        leftmost_four = key >> 76
        sboxed_leftmost_four = s_box[leftmost_four] << 76 
        rightmost_76 = (key & int('1' * 76, base=2))
        key = sboxed_leftmost_four + rightmost_76

        # the round_counter value i is exclusive-ored with bits k19 k18 k17 k16 k15 
        # of K with the least significant bit of round_counter on the right
        key ^= i << 15 
    return roundkeys

#############################################

import numpy as np

def Sbox(x):
    temp_x = x & 0b1111
    temp_y = x >> 4

    table = [0xc, 0x5, 0x6, 0xb, 0x9, 0x00, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2]
    
    temp_x = table[temp_x]
    temp_y = table[temp_y]

    temp = [0,0,0,0]
    temp[0] = (temp_y & 0b0001)
    temp[1] = (temp_y & 0b0010) >> 1
    temp[2] = (temp_y & 0b0100) >> 2
    temp[3] = (temp_y & 0b1000) >> 3

    result = temp_x + temp[0] * 16 +  temp[1] * 32 + temp[2] * 64 + temp[3] * 128

    return result

def Permutation(x):

    # 73625140
    temp = [0,0,0,0,0,0,0,0]
    temp[0] = x & 0b0001
    temp[1] = (x & 0b0010) >> 1
    temp[2] = (x & 0b0100) >> 2
    temp[3] = (x & 0b1000) >> 3

    temp[4] = (x & 0b00010000) >> 4
    temp[5] = (x & 0b00100000) >> 5
    temp[6] = (x & 0b01000000) >> 6
    temp[7] = (x & 0b10000000) >> 7

    result = 0
    result = result + temp[0]
    result = result + temp[4] * 2
    result = result + temp[1] * 4
    result = result + temp[5] * 8
    result = result + temp[2] * 16
    result = result + temp[6] * 32
    result = result + temp[3] * 64
    result = result + temp[7] * 128

    return result

'''
for i in range(rounds-1):
        state = addRoundKey(state, roundkeys[i])
        state = sBoxLayer(state)
        state = pLayer(state)
    state = addRoundKey(state, roundkeys[rounds-1])

'''
def Present(pt0, key, rounds):
    n = 2
    s_roundkeys = [0,0,0,0,0,0,0,0,0,0,0]
    roundkeys = genRoundKeys(key, rounds)
    
    for rnd in range(len(roundkeys)):         
        intrnd = bin(roundkeys[rnd])[-4*n:] 
        s_roundkeys[rnd] = int(intrnd,2)
        #print(s_roundkeys)
    
    for i in range(rounds-1):
        pt0 = pt0 ^ s_roundkeys[i]
        pt0 = Sbox(pt0)
        pt0 = Permutation(pt0)
    ct0 = pt0 ^ s_roundkeys[rounds-1]

    return ct0

In [ ]:
plain = 0
key = 0

In [ ]:
ctdata = Present(plain, key, 11) # Round +1 해서 보기

0
240
89
85
98
182
247
2
248
137


In [ ]:
hex(ctdata)

'0xb'